# Survey methods examples
This notebook contains examples of the following methods for adding rules/conditional logic and question memories to a survey:

* `add_skip_rule()` - Skip a question based on a conditional expression (e.g., the response to another question).
* `add_stop_rule()` - End the survey based on a conditional expression.
* `add_rule()` - Administer a specified question next based on a conditional expression.
* `set_full_memory_mode()` - Include a memory of all prior questions/answers at each new question in the survey.
* `set_lagged_memory()` - Include a memory of a specified number of prior questions/answers at each new question in the survey.
* `add_targeted_memory()` - Include a memory of a specific question/answer at another question in the survey.
* `add_memory_collection()` - Include memories of a set of prior questions/answers at any other question in the survey.

See the [EDSL Docs](https://docs.expectedparrot.com/en/latest/surveys.html) for more details on these and other methods.

## Example survey
We start by creating some questions to demonstrate the methods.

In [1]:
from edsl.questions import QuestionMultipleChoice, QuestionLinearScale, QuestionTopK
from edsl import Survey

q1 = QuestionMultipleChoice(
    question_name = "color",
    question_text = "What is your favorite color?",
    question_options = ["Red", "Orange", "Yellow", "Green", "Blue", "Purple"]
)
q2 = QuestionMultipleChoice(
    question_name = "day",
    question_text = "What is your favorite day of the week?",
    question_options = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
)
q3 = QuestionLinearScale(
    question_name = "winter",
    question_text = "How much do you enjoy winter?",
    question_options = [0,1,2,3,4,5],
    option_labels = {0: "Hate it", 5: "Love it"}
)
q4 = QuestionTopK(
    question_name = "birds",
    question_text = "Which birds do you like best?",
    question_options = ["Parrot", "Osprey", "Falcon", "Eagle", "First Robin of Spring"],
    min_selections = 2,
    max_selections = 2
)

## Skip rules
The `add_skip_rule()` method skips a question if a condition is met. The (2) required parameters are the question to skip and the condition to evaluate.

Here we use this method to skip q2 if the response to “color” is “Blue”. Note that we can refer to the question to be skipped using either the id (“q2”) or question_name (“day”):

In [2]:
survey = Survey(questions = [q1, q2, q3, q4])
survey = survey.add_skip_rule(q2, "color == 'Blue'")

results = survey.run()
results.select("color", "day", "winter", "birds").print(format="rich")

┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ answer ┃ answer ┃ answer  ┃ answer              ┃
┃ .color ┃ .day   ┃ .winter ┃ .birds              ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ Blue   │ None   │ 3       │ ['Falcon', 'Eagle'] │
└────────┴────────┴─────────┴─────────────────────┘

We can see that the response to q2 is "None" indicating that the question was not administered to the agent.

## Stop rules
The `add_stop_rule()` method stops the survey if a condition is met. The (2) required parameters are the question to stop at and the condition to evaluate.

Here we use the method to end the survey at q1 if the response is "Blue":

In [3]:
survey = Survey(questions = [q1, q2, q3, q4])
survey = survey.add_stop_rule(q1, "color == 'Blue'")

results = survey.run()
results.select("color", "day", "winter", "birds").print(format="rich")

┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ answer ┃ answer ┃ answer  ┃ answer ┃
┃ .color ┃ .day   ┃ .winter ┃ .birds ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ Blue   │ None   │ None    │ None   │
└────────┴────────┴─────────┴────────┘

## Other rules
The generalizable `add_rule()` method is used to specify the next question to administer based on a condition. The (3) required parameters are the question to evaluate, the condition to evaluate, and the question to administer next.

Here we the method to specify that if the response to “color” is “Blue” then q4 should be administered next:

In [4]:
survey = Survey(questions = [q1, q2, q3, q4])
survey = survey.add_rule(q1, "color == 'Blue'", q4)

results = survey.run()
results.select("color", "day", "winter", "birds").print(format="rich")

┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ answer ┃ answer ┃ answer  ┃ answer              ┃
┃ .color ┃ .day   ┃ .winter ┃ .birds              ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ Blue   │ None   │ None    │ ['Falcon', 'Eagle'] │
└────────┴────────┴─────────┴─────────────────────┘

## Question memories
When an agent is taking a survey, they can be prompted to “remember” answers to previous questions. This can be done in several ways:

## Full memory
The method `set_full_memory_mode()` gives the agent all of the prior questions and answers at each new question in the survey, i.e., the first question and answer are included in the memory when answering the second question, both the first and second questions and answers are included in the memory when answering the third question, and so on. The method is called on the survey object:

In [5]:
survey = Survey(questions = [q1, q2, q3, q4])
survey = survey.set_full_memory_mode()

In the results, we can inspect the `_user_prompt` for each question to see that the agent was prompted to remember all of the prior questions:

In [6]:
results = survey.run()
results.select("color_user_prompt", "day_user_prompt", "winter_user_prompt", "birds_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                     ┃ prompt                    ┃ prompt                     ┃ prompt                    ┃
┃ .color_user_prompt         ┃ .day_user_prompt          ┃ .winter_user_prompt        ┃ .birds_user_prompt        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ {'text': 'You are being    │ {'text': 'You are being   │ {'text': 'You are being    │ {'text': 'You are being   │
│ asked the following        │ asked the following       │ asked the following        │ asked the following       │
│ question: What is your     │ question: What is your    │ question: How much do you  │ question: Which birds do  │
│ favorite color?\nThe       │ favorite day of the       │ enjoy winter?\nThe options │ you like best?\nThe       │
│ options are\n\n0:          │ week?\nThe options        │ are\n\n0: 0\n\n1: 1\n\n2:  │ options are\n\n0:         │
│ Red\n\n1: Orange\n\n2:     │ are\n\n0: Sun\n\n1:       │ 2\n\n3: 3\n\n4: 4\n\n5:    │ Parrot\n\n1: Osprey\n\n2: │
│ Yellow\n\n3: Green\n\n4:   │ Mon\n\n2: Tue\n\n3:       │ 5\n\nReturn a valid JSON   │ Falcon\n\n3: Eagle\n\n4:  │
│ Blue\n\n5:                 │ Wed\n\n4: Thu\n\n5:       │ formatted like this,       │ First Robin of            │
│ Purple\n\nReturn a valid   │ Fri\n\n6: Sat\n\nReturn a │ selecting only the code of │ Spring\n\nReturn a valid  │
│ JSON formatted like this,  │ valid JSON formatted like │ the option (codes start at │ JSON formatted like this, │
│ selecting only the number  │ this, selecting only the  │ 0):\n{"answer": <put       │ selecting only the number │
│ of the option:\n{"answer": │ number of the             │ answer code here>,         │ of the                    │
│ <put answer code here>,    │ option:\n{"answer": <put  │ "comment": "<put           │ option:\n{"answer": [<put │
│ "comment": "<put           │ answer code here>,        │ explanation here>"}\nOnly  │ comma-separated list of   │
│ explanation here>"}\nOnly  │ "comment": "<put          │ 1 option may be            │ answer codes here>],      │
│ 1 option may be            │ explanation here>"}\nOnly │ selected.\n        Before  │ "comment": "<put          │
│ selected.', 'class_name':  │ 1 option may be           │ the question you are now   │ explanation               │
│ 'MultipleChoice'}          │ selected.\n        Before │ answering, you already     │ here>"}\n\nYou must       │
│                            │ the question you are now  │ answered the following     │ select exactly 2          │
│                            │ answering, you already    │ question(s):\n             │ options.\n        Before  │
│                            │ answered the following    │ \tQuestion: What is your   │ the question you are now  │
│                            │ question(s):\n            │ favorite color?\n\tAnswer: │ answering, you already    │
│                            │ \tQuestion: What is your  │ Blue\n\n Prior questions   │ answered the following    │
│                            │ favorite                  │ and answers:\tQuestion:    │ question(s):\n            │
│                            │ color?\n\tAnswer: Blue',  │ What is your favorite day  │ \tQuestion: What is your  │
│                            │ 'class_name':             │ of the week?\n\tAnswer:    │ favorite                  │
│                            │ 'MultipleChoice'}         │ Fri', 'class_name':        │ color?\n\tAnswer:         │
│                            │                           │ 'LinearScale'}             │ Blue\n\n Prior questions  │
│                            │                           │                            │ and answers:\tQuestion:   │
│                            │                           │                            │ What is your favorite day │
│                            │                          

## Lagged memory
The method `set_lagged_memory()` gives the agent a specified number of prior questions and answers at each new question in the survey; we pass it the number of prior questions and answers to remember. Here we use it to give the agent just 1 prior question/answer at each question:

In [7]:
survey = Survey(questions = [q1, q2, q3, q4])
survey = survey.set_lagged_memory(1)

We can inspect each _user_prompt again and see that the agent is only prompted to remember the last prior question/answer:

In [8]:
results = survey.run()
results.select("color_user_prompt", "day_user_prompt", "winter_user_prompt", "birds_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                     ┃ prompt                    ┃ prompt                     ┃ prompt                    ┃
┃ .color_user_prompt         ┃ .day_user_prompt          ┃ .winter_user_prompt        ┃ .birds_user_prompt        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ {'text': 'You are being    │ {'text': 'You are being   │ {'text': 'You are being    │ {'text': 'You are being   │
│ asked the following        │ asked the following       │ asked the following        │ asked the following       │
│ question: What is your     │ question: What is your    │ question: How much do you  │ question: Which birds do  │
│ favorite color?\nThe       │ favorite day of the       │ enjoy winter?\nThe options │ you like best?\nThe       │
│ options are\n\n0:          │ week?\nThe options        │ are\n\n0: 0\n\n1: 1\n\n2:  │ options are\n\n0:         │
│ Red\n\n1: Orange\n\n2:     │ are\n\n0: Sun\n\n1:       │ 2\n\n3: 3\n\n4: 4\n\n5:    │ Parrot\n\n1: Osprey\n\n2: │
│ Yellow\n\n3: Green\n\n4:   │ Mon\n\n2: Tue\n\n3:       │ 5\n\nReturn a valid JSON   │ Falcon\n\n3: Eagle\n\n4:  │
│ Blue\n\n5:                 │ Wed\n\n4: Thu\n\n5:       │ formatted like this,       │ First Robin of            │
│ Purple\n\nReturn a valid   │ Fri\n\n6: Sat\n\nReturn a │ selecting only the code of │ Spring\n\nReturn a valid  │
│ JSON formatted like this,  │ valid JSON formatted like │ the option (codes start at │ JSON formatted like this, │
│ selecting only the number  │ this, selecting only the  │ 0):\n{"answer": <put       │ selecting only the number │
│ of the option:\n{"answer": │ number of the             │ answer code here>,         │ of the                    │
│ <put answer code here>,    │ option:\n{"answer": <put  │ "comment": "<put           │ option:\n{"answer": [<put │
│ "comment": "<put           │ answer code here>,        │ explanation here>"}\nOnly  │ comma-separated list of   │
│ explanation here>"}\nOnly  │ "comment": "<put          │ 1 option may be            │ answer codes here>],      │
│ 1 option may be            │ explanation here>"}\nOnly │ selected.\n        Before  │ "comment": "<put          │
│ selected.', 'class_name':  │ 1 option may be           │ the question you are now   │ explanation               │
│ 'MultipleChoice'}          │ selected.\n        Before │ answering, you already     │ here>"}\n\nYou must       │
│                            │ the question you are now  │ answered the following     │ select exactly 2          │
│                            │ answering, you already    │ question(s):\n             │ options.\n        Before  │
│                            │ answered the following    │ \tQuestion: What is your   │ the question you are now  │
│                            │ question(s):\n            │ favorite day of the        │ answering, you already    │
│                            │ \tQuestion: What is your  │ week?\n\tAnswer: Fri',     │ answered the following    │
│                            │ favorite                  │ 'class_name':              │ question(s):\n            │
│                            │ color?\n\tAnswer: Blue',  │ 'LinearScale'}             │ \tQuestion: How much do   │
│                            │ 'class_name':             │                            │ you enjoy                 │
│                            │ 'MultipleChoice'}         │                            │ winter?\n\tAnswer: 0',    │
│                            │                           │                            │ 'class_name': 'TopK'}     │
└────────────────────────────┴───────────────────────────┴────────────────────────────┴───────────────────────────┘

## Targeted memory
The method `add_targeted_memory()` gives the agent a targeted prior question and answer when answering another specified question. We pass it the question to answer and the prior question/answer to remember when answering it. Here we use it to give the agent the question/answer to q1 when prompting it to answer q4:

In [9]:
survey = Survey(questions = [q1, q2, q3, q4])
survey = survey.add_targeted_memory(q4, q1)

In [10]:
results = survey.run()
results.select("color_user_prompt", "day_user_prompt", "winter_user_prompt", "birds_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                     ┃ prompt                    ┃ prompt                     ┃ prompt                    ┃
┃ .color_user_prompt         ┃ .day_user_prompt          ┃ .winter_user_prompt        ┃ .birds_user_prompt        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ {'text': 'You are being    │ {'text': 'You are being   │ {'text': 'You are being    │ {'text': 'You are being   │
│ asked the following        │ asked the following       │ asked the following        │ asked the following       │
│ question: What is your     │ question: What is your    │ question: How much do you  │ question: Which birds do  │
│ favorite color?\nThe       │ favorite day of the       │ enjoy winter?\nThe options │ you like best?\nThe       │
│ options are\n\n0:          │ week?\nThe options        │ are\n\n0: 0\n\n1: 1\n\n2:  │ options are\n\n0:         │
│ Red\n\n1: Orange\n\n2:     │ are\n\n0: Sun\n\n1:       │ 2\n\n3: 3\n\n4: 4\n\n5:    │ Parrot\n\n1: Osprey\n\n2: │
│ Yellow\n\n3: Green\n\n4:   │ Mon\n\n2: Tue\n\n3:       │ 5\n\nReturn a valid JSON   │ Falcon\n\n3: Eagle\n\n4:  │
│ Blue\n\n5:                 │ Wed\n\n4: Thu\n\n5:       │ formatted like this,       │ First Robin of            │
│ Purple\n\nReturn a valid   │ Fri\n\n6: Sat\n\nReturn a │ selecting only the code of │ Spring\n\nReturn a valid  │
│ JSON formatted like this,  │ valid JSON formatted like │ the option (codes start at │ JSON formatted like this, │
│ selecting only the number  │ this, selecting only the  │ 0):\n{"answer": <put       │ selecting only the number │
│ of the option:\n{"answer": │ number of the             │ answer code here>,         │ of the                    │
│ <put answer code here>,    │ option:\n{"answer": <put  │ "comment": "<put           │ option:\n{"answer": [<put │
│ "comment": "<put           │ answer code here>,        │ explanation here>"}\nOnly  │ comma-separated list of   │
│ explanation here>"}\nOnly  │ "comment": "<put          │ 1 option may be            │ answer codes here>],      │
│ 1 option may be            │ explanation here>"}\nOnly │ selected.', 'class_name':  │ "comment": "<put          │
│ selected.', 'class_name':  │ 1 option may be           │ 'LinearScale'}             │ explanation               │
│ 'MultipleChoice'}          │ selected.', 'class_name': │                            │ here>"}\n\nYou must       │
│                            │ 'MultipleChoice'}         │                            │ select exactly 2          │
│                            │                           │                            │ options.\n        Before  │
│                            │                           │                            │ the question you are now  │
│                            │                           │                            │ answering, you already    │
│                            │                           │                            │ answered the following    │
│                            │                           │                            │ question(s):\n            │
│                            │                           │                            │ \tQuestion: What is your  │
│                            │                           │                            │ favorite                  │
│                            │                           │                            │ color?\n\tAnswer: Blue',  │
│                            │                           │                            │ 'class_name': 'TopK'}     │
└────────────────────────────┴───────────────────────────┴────────────────────────────┴───────────────────────────┘

## Memory collection
The `add_memory_collection()` method is used to add sets of prior questions and answers to a given question. We pass it the question to be answered and the list of questions/answers to be remembered when answering it. For example, we can add the questions/answers for both q1 and q2 when prompting the agent to answer q4:

In [11]:
survey = Survey(questions = [q1, q2, q3, q4])
survey = survey.add_memory_collection(q4, [q1, q2])

In [12]:
results = survey.run()
results.select("color_user_prompt", "day_user_prompt", "winter_user_prompt", "birds_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                     ┃ prompt                    ┃ prompt                     ┃ prompt                    ┃
┃ .color_user_prompt         ┃ .day_user_prompt          ┃ .winter_user_prompt        ┃ .birds_user_prompt        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ {'text': 'You are being    │ {'text': 'You are being   │ {'text': 'You are being    │ {'text': 'You are being   │
│ asked the following        │ asked the following       │ asked the following        │ asked the following       │
│ question: What is your     │ question: What is your    │ question: How much do you  │ question: Which birds do  │
│ favorite color?\nThe       │ favorite day of the       │ enjoy winter?\nThe options │ you like best?\nThe       │
│ options are\n\n0:          │ week?\nThe options        │ are\n\n0: 0\n\n1: 1\n\n2:  │ options are\n\n0:         │
│ Red\n\n1: Orange\n\n2:     │ are\n\n0: Sun\n\n1:       │ 2\n\n3: 3\n\n4: 4\n\n5:    │ Parrot\n\n1: Osprey\n\n2: │
│ Yellow\n\n3: Green\n\n4:   │ Mon\n\n2: Tue\n\n3:       │ 5\n\nReturn a valid JSON   │ Falcon\n\n3: Eagle\n\n4:  │
│ Blue\n\n5:                 │ Wed\n\n4: Thu\n\n5:       │ formatted like this,       │ First Robin of            │
│ Purple\n\nReturn a valid   │ Fri\n\n6: Sat\n\nReturn a │ selecting only the code of │ Spring\n\nReturn a valid  │
│ JSON formatted like this,  │ valid JSON formatted like │ the option (codes start at │ JSON formatted like this, │
│ selecting only the number  │ this, selecting only the  │ 0):\n{"answer": <put       │ selecting only the number │
│ of the option:\n{"answer": │ number of the             │ answer code here>,         │ of the                    │
│ <put answer code here>,    │ option:\n{"answer": <put  │ "comment": "<put           │ option:\n{"answer": [<put │
│ "comment": "<put           │ answer code here>,        │ explanation here>"}\nOnly  │ comma-separated list of   │
│ explanation here>"}\nOnly  │ "comment": "<put          │ 1 option may be            │ answer codes here>],      │
│ 1 option may be            │ explanation here>"}\nOnly │ selected.', 'class_name':  │ "comment": "<put          │
│ selected.', 'class_name':  │ 1 option may be           │ 'LinearScale'}             │ explanation               │
│ 'MultipleChoice'}          │ selected.', 'class_name': │                            │ here>"}\n\nYou must       │
│                            │ 'MultipleChoice'}         │                            │ select exactly 2          │
│                            │                           │                            │ options.\n        Before  │
│                            │                           │                            │ the question you are now  │
│                            │                           │                            │ answering, you already    │
│                            │                           │                            │ answered the following    │
│                            │                           │                            │ question(s):\n            │
│                            │                           │                            │ \tQuestion: What is your  │
│                            │                           │                            │ favorite                  │
│                            │                           │                            │ color?\n\tAnswer:         │
│                            │                           │                            │ Blue\n\n Prior questions  │
│                            │                           │                            │ and answers:\tQuestion:   │
│                            │                           │                            │ What is your favorite day │
│                            │                          